In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV    
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression

In [3]:
## Dimensões para visualização de datasets
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.8f}'.format)

In [4]:
train = pd.read_csv('train.csv')
train.head()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2t_mou_8,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2m_mou_8,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2f_mou_8,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_t2c_mou_8,loc_og_mou_6,loc_og_mou_7,loc_og_mou_8,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2t_mou_8,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2m_mou_8,std_og_t2f_mou_6,std_og_t2f_mou_7,std_og_t2f_mou_8,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_t2c_mou_8,std_og_mou_6,std_og_mou_7,std_og_mou_8,isd_og_mou_6,isd_og_mou_7,isd_og_mou_8,spl_og_mou_6,spl_og_mou_7,spl_og_mou_8,og_others_6,og_others_7,og_others_8,total_og_mou_6,total_og_mou_7,total_og_mou_8,loc_ic_t2t_mou_6,loc_ic_t2t_mou_7,loc_ic_t2t_mou_8,loc_ic_t2m_mou_6,loc_ic_t2m_mou_7,loc_ic_t2m_mou_8,loc_ic_t2f_mou_6,loc_ic_t2f_mou_7,loc_ic_t2f_mou_8,loc_ic_mou_6,loc_ic_mou_7,loc_ic_mou_8,std_ic_t2t_mou_6,std_ic_t2t_mou_7,std_ic_t2t_mou_8,std_ic_t2m_mou_6,std_ic_t2m_mou_7,std_ic_t2m_mou_8,std_ic_t2f_mou_6,std_ic_t2f_mou_7,std_ic_t2f_mou_8,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_t2o_mou_8,std_ic_mou_6,std_ic_mou_7,std_ic_mou_8,total_ic_mou_6,total_ic_mou_7,total_ic_mou_8,spl_ic_mou_6,spl_ic_mou_7,spl_ic_mou_8,isd_ic_mou_6,isd_ic_mou_7,isd_ic_mou_8,ic_others_6,ic_others_7,ic_others_8,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8,total_rech_data_6,total_rech_data_7,total_rech_data_8,max_rech_data_6,max_rech_data_7,max_rech_data_8,count_rech_2g_6,count_rech_2g_7,count_rech_2g_8,count_rech_3g_6,count_rech_3g_7,count_rech_3g_8,av_rech_amt_data_6,av_rech_amt_data_7,av_rech_amt_data_8,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,arpu_3g_6,arpu_3g_7,arpu_3g_8,arpu_2g_6,arpu_2g_7,arpu_2g_8,night_pck_user_6,night_pck_user_7,night_pck_user_8,monthly_2g_6,monthly_2g_7,monthly_2g_8,sachet_2g_6,sachet_2g_7,sachet_2g_8,monthly_3g_6,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
0,0,109,0.00000000,0.00000000,0.00000000,6/30/2014,7/31/2014,8/31/2014,31.27700000,87.00900000,7.52700000,48.58000000,124.38000000,1.29000000,32.24000000,96.68000000,2.33000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,2.23000000,0.00000000,0.28000000,5.29000000,16.04000000,2.33000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,7.53000000,16.04000000,2.61000000,46.34000000,124.38000000,1.01000000,18.75000000,80.61000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,65.09000000,204.99000000,1.01000000,0.00000000,0.00000000,0.00000000,8.20000000,0.63000000,0.00000000,0.38000000,0.00000000,0.00000000,81.21000000,221.68000000,3.63000000,2.43000000,3.68000000,7.79000000,0.83000000,21.08000000,16.91000000,0.00000000,0.00000000,0.00000000,3.26000000,24.76000000,24.71000000,0.00000000,7.61000000,0.21000000,7.46000000,19.96000000,14.96000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,7.46000000,27.58000000,15.18000000,11.84000000,53.04000000,40.56000000,0.00000000,0.00000000,0.66000000,0.00000000,0.00000000,0.00000000,1.11000000,0.69000000,0.00000000,3,2,2,77,65,10,65,65,10,6/22/2014,7/10/2014,8/24/2014,65,65,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,1958,0.00000000,0

def aon_fx1(x):
    if x <= 30:
        return 0
    elif x > 30 and x <= 60:
        return 1
    elif x > 60 and x <= 90:
        return 2
    elif x > 90 and x <= 120:
        return 3
    elif x > 120 and x <= 365:
        return 4
    elif x > 365 and x <= 730:
        return 5
    elif x > 730:
        return 6
    else:
        return 7

train['aon_fx'] = train.aon.apply(aon_fx1)
    
train.aon_fx.value_counts()

In [4]:
train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69999 entries, 0 to 69998
Data columns (total 172 columns):
 #    Column                    Dtype  
---   ------                    -----  
 0    id                        int64  
 1    circle_id                 int64  
 2    loc_og_t2o_mou            float64
 3    std_og_t2o_mou            float64
 4    loc_ic_t2o_mou            float64
 5    last_date_of_month_6      object 
 6    last_date_of_month_7      object 
 7    last_date_of_month_8      object 
 8    arpu_6                    float64
 9    arpu_7                    float64
 10   arpu_8                    float64
 11   onnet_mou_6               float64
 12   onnet_mou_7               float64
 13   onnet_mou_8               float64
 14   offnet_mou_6              float64
 15   offnet_mou_7              float64
 16   offnet_mou_8              float64
 17   roam_ic_mou_6             float64
 18   roam_ic_mou_7             float64
 19   roam_ic_mou_8             float64
 20   roam

In [5]:
features_names = [
"loc_og_t2o_mou",
"std_og_t2o_mou",
"loc_ic_t2o_mou",
"arpu_6",
"arpu_7",
"arpu_8",
"onnet_mou_6",
"onnet_mou_7",
"onnet_mou_8",
"offnet_mou_6",
"offnet_mou_7",
"offnet_mou_8",
"roam_ic_mou_6",
"roam_ic_mou_7",
"roam_ic_mou_8",
"roam_og_mou_6",
"roam_og_mou_7",
"roam_og_mou_8",
"loc_og_t2t_mou_6",
"loc_og_t2t_mou_7",
"loc_og_t2t_mou_8",
"loc_og_t2m_mou_6",
"loc_og_t2m_mou_7",
"loc_og_t2m_mou_8",
"loc_og_t2f_mou_6",
"loc_og_t2f_mou_7",
"loc_og_t2f_mou_8",
"loc_og_t2c_mou_6",
"loc_og_t2c_mou_7",
"loc_og_t2c_mou_8",
"loc_og_mou_6",
"loc_og_mou_7",
"loc_og_mou_8",
"std_og_t2t_mou_6",
"std_og_t2t_mou_7",
"std_og_t2t_mou_8",
"std_og_t2m_mou_6",
"std_og_t2m_mou_7",
"std_og_t2m_mou_8",
"std_og_t2f_mou_6",
"std_og_t2f_mou_7",
"std_og_t2f_mou_8",
"std_og_t2c_mou_6",
"std_og_t2c_mou_7",
"std_og_t2c_mou_8",
"std_og_mou_6",
"std_og_mou_7",
"std_og_mou_8",
"isd_og_mou_6",
"isd_og_mou_7",
"isd_og_mou_8",
"spl_og_mou_6",
"spl_og_mou_7",
"spl_og_mou_8",
"og_others_6",
"og_others_7",
"og_others_8",
"total_og_mou_6",
"total_og_mou_7",
"total_og_mou_8",
"loc_ic_t2t_mou_6",
"loc_ic_t2t_mou_7",
"loc_ic_t2t_mou_8",
"loc_ic_t2m_mou_6",
"loc_ic_t2m_mou_7",
"loc_ic_t2m_mou_8",
"loc_ic_t2f_mou_6",
"loc_ic_t2f_mou_7",
"loc_ic_t2f_mou_8",
"loc_ic_mou_6",
"loc_ic_mou_7",
"loc_ic_mou_8",
"std_ic_t2t_mou_6",
"std_ic_t2t_mou_7",
"std_ic_t2t_mou_8",
"std_ic_t2m_mou_6",
"std_ic_t2m_mou_7",
"std_ic_t2m_mou_8",
"std_ic_t2f_mou_6",
"std_ic_t2f_mou_7",
"std_ic_t2f_mou_8",
"std_ic_t2o_mou_6",
"std_ic_t2o_mou_7",
"std_ic_t2o_mou_8",
"std_ic_mou_6",
"std_ic_mou_7",
"std_ic_mou_8",
"total_ic_mou_6",
"total_ic_mou_7",
"total_ic_mou_8",
"spl_ic_mou_6",
"spl_ic_mou_7",
"spl_ic_mou_8",
"isd_ic_mou_6",
"isd_ic_mou_7",
"isd_ic_mou_8",
"ic_others_6",
"ic_others_7",
"ic_others_8",
"total_rech_num_6",
"total_rech_num_7",
"total_rech_num_8",
"total_rech_amt_6",
"total_rech_amt_7",
"total_rech_amt_8",
"max_rech_amt_6",
"max_rech_amt_7",
"max_rech_amt_8",
"last_day_rch_amt_6",
"last_day_rch_amt_7",
"last_day_rch_amt_8",
"total_rech_data_6",
"total_rech_data_7",
"total_rech_data_8",
"max_rech_data_6",
"max_rech_data_7",
"max_rech_data_8",
"count_rech_2g_6",
"count_rech_2g_7",
"count_rech_2g_8",
"count_rech_3g_6",
"count_rech_3g_7",
"count_rech_3g_8",
"av_rech_amt_data_6",
"av_rech_amt_data_7",
"av_rech_amt_data_8",
"vol_2g_mb_6",
"vol_2g_mb_7",
"vol_2g_mb_8",
"vol_3g_mb_6",
"vol_3g_mb_7",
"vol_3g_mb_8",
"arpu_3g_6",
"arpu_3g_7",
"arpu_3g_8",
"arpu_2g_6",
"arpu_2g_7",
"arpu_2g_8",
"night_pck_user_6",
"night_pck_user_7",
"night_pck_user_8",
"monthly_2g_6",
"monthly_2g_7",
"monthly_2g_8",
"sachet_2g_6",
"sachet_2g_7",
"sachet_2g_8",
"monthly_3g_6",
"monthly_3g_7",
"monthly_3g_8",
"sachet_3g_6",
"sachet_3g_7",
"sachet_3g_8",
"fb_user_6",
"fb_user_7",
"fb_user_8",
"aon",
"aug_vbc_3g",
"jul_vbc_3g",
"jun_vbc_3g"
    
]

In [6]:
X_train = train.set_index('id')[features_names].fillna(train.mean())
X_train.head()

,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2t_mou_8,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2m_mou_8,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2f_mou_8,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_t2c_mou_8,loc_og_mou_6,loc_og_mou_7,loc_og_mou_8,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2t_mou_8,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2m_mou_8,std_og_t2f_mou_6,std_og_t2f_mou_7,std_og_t2f_mou_8,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_t2c_mou_8,std_og_mou_6,std_og_mou_7,std_og_mou_8,isd_og_mou_6,isd_og_mou_7,isd_og_mou_8,spl_og_mou_6,spl_og_mou_7,spl_og_mou_8,og_others_6,og_others_7,og_others_8,total_og_mou_6,total_og_mou_7,total_og_mou_8,loc_ic_t2t_mou_6,loc_ic_t2t_mou_7,loc_ic_t2t_mou_8,loc_ic_t2m_mou_6,loc_ic_t2m_mou_7,loc_ic_t2m_mou_8,loc_ic_t2f_mou_6,loc_ic_t2f_mou_7,loc_ic_t2f_mou_8,loc_ic_mou_6,loc_ic_mou_7,loc_ic_mou_8,std_ic_t2t_mou_6,std_ic_t2t_mou_7,std_ic_t2t_mou_8,std_ic_t2m_mou_6,std_ic_t2m_mou_7,std_ic_t2m_mou_8,std_ic_t2f_mou_6,std_ic_t2f_mou_7,std_ic_t2f_mou_8,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_t2o_mou_8,std_ic_mou_6,std_ic_mou_7,std_ic_mou_8,total_ic_mou_6,total_ic_mou_7,total_ic_mou_8,spl_ic_mou_6,spl_ic_mou_7,spl_ic_mou_8,isd_ic_mou_6,isd_ic_mou_7,isd_ic_mou_8,ic_others_6,ic_others_7,ic_others_8,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,total_rech_data_6,total_rech_data_7,total_rech_data_8,max_rech_data_6,max_rech_data_7,max_rech_data_8,count_rech_2g_6,count_rech_2g_7,count_rech_2g_8,count_rech_3g_6,count_rech_3g_7,count_rech_3g_8,av_rech_amt_data_6,av_rech_amt_data_7,av_rech_amt_data_8,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,arpu_3g_6,arpu_3g_7,arpu_3g_8,arpu_2g_6,arpu_2g_7,arpu_2g_8,night_pck_user_6,night_pck_user_7,night_pck_user_8,monthly_2g_6,monthly_2g_7,monthly_2g_8,sachet_2g_6,sachet_2g_7,sachet_2g_8,monthly_3g_6,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.00000000,0.00000000,0.00000000,31.27700000,87.00900000,7.52700000,48.58000000,124.38000000,1.29000000,32.24000000,96.68000000,2.33000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,2.23000000,0.00000000,0.28000000,5.29000000,16.04000000,2.33000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,7.53000000,16.04000000,2.61000000,46.34000000,124.38000000,1.01000000,18.75000000,80.61000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,65.09000000,204.99000000,1.01000000,0.00000000,0.00000000,0.00000000,8.20000000,0.63000000,0.00000000,0.38000000,0.00000000,0.00000000,81.21000000,221.68000000,3.63000000,2.43000000,3.68000000,7.79000000,0.83000000,21.08000000,16.91000000,0.00000000,0.00000000,0.00000000,3.26000000,24.76000000,24.71000000,0.00000000,7.61000000,0.21000000,7.46000000,19.96000000,14.96000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,7.46000000,27.58000000,15.18000000,11.84000000,53.04000000,40.56000000,0.00000000,0.00000000,0.66000000,0.00000000,0.00000000,0.00000000,1.11000000,0.69000000,0.00000000,3,2,2,77,65,10,65,65,10,65,65,0,2.46761157,2.67998880,2.65244068,126.50000000,126.40207109,125.37492534,1.86532332,2.05631122,2.01601781,0.60228825,0.62367758,0.63642287,192.83109574,201.45594011,196.81579247,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,90.06993112,89.11576714,90.61856437,86.86390027,85.84607445,86.34840365,0.02527322,0.02406941,0.02101319,0,0,0,0,0,0

In [7]:
scaler = StandardScaler()
X = scaler.fit_transform(X_train)

In [8]:
X

array([[ 0.        ,  0.        ,  0.        , ..., -0.25288465,
        -0.2461237 , -0.23355718],
       [ 0.        ,  0.        ,  0.        , ..., -0.25288465,
        -0.2461237 , -0.23355718],
       [ 0.        ,  0.        ,  0.        , ..., -0.25288465,
        -0.2461237 , -0.23355718],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., -0.25288465,
        -0.2461237 , -0.23355718],
       [ 0.        ,  0.        ,  0.        , ...,  2.21345052,
         4.99081204,  3.61898477],
       [ 0.        ,  0.        ,  0.        , ..., -0.25288465,
        -0.2461237 , -0.23355718]])

In [9]:
y = train['churn_probability']
y

0        0
1        0
2        0
3        0
4        0
        ..
69994    0
69995    0
69996    0
69997    0
69998    0
Name: churn_probability, Length: 69999, dtype: int64

In [10]:
random_grid = {'n_estimators': [ 100,  307,  514,  721,  928, 1135, 1342, 1550, 1757, 1964, 2171,2378, 2585, 2792, 3000],  
 'max_depth': [1, 5, 10, 20, 50, 75, 100, 150, 200],  
 'min_samples_split': [1, 2, 5, 10, 15, 20, 30],  
 'min_samples_leaf': [1, 2, 3, 4],  
 'bootstrap': [True, False],  
 'criterion': ['gini', 'entropy']}

In [12]:
%%time
clf = RandomForestClassifier() 
clf_random = RandomizedSearchCV(estimator = clf,
                         param_distributions = random_grid,
                         n_iter = 30, cv = 5,
                         verbose=2,
                         random_state=42,
                         n_jobs = 4)
clf_random.fit(X,y)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


c:\users\lincoln.thiago.neves\appdata\local\programs\python\python39\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "c:\users\lincoln.thiago.neves\appdata\local\programs\python\python39\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\lincoln.thiago.neves\appdata\local\programs\python\python39\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "c:\users\lincoln.thiago.neves\appdata\local\progr

Wall time: 11h 47min 28s


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=30,
                   n_jobs=4,
                   param_distributions={'bootstrap': [True, False],
                                        'criterion': ['gini', 'entropy'],
                                        'max_depth': [1, 5, 10, 20, 50, 75, 100,
                                                      150, 200],
                                        'min_samples_leaf': [1, 2, 3, 4],
                                        'min_samples_split': [1, 2, 5, 10, 15,
                                                              20, 30],
                                        'n_estimators': [100, 307, 514, 721,
                                                         928, 1135, 1342, 1550,
                                                         1757, 1964, 2171, 2378,
                                                         2585, 2792, 3000]},
                   random_state=42, verbose=2)

In [13]:
clf_random.best_params_

{'n_estimators': 307,
 'min_samples_split': 15,
 'min_samples_leaf': 3,
 'max_depth': 20,
 'criterion': 'entropy',
 'bootstrap': True}

In [ ]:
param_grid = {
    'n_estimators': np.linspace(300, 350, 5, dtype = int),
    'max_depth': [5,10,15,20,25,30],  
    'min_samples_split': [15,16,17],  
    'min_samples_leaf': [3,4,5,6]}

In [ ]:
%%time
clf_grid = RandomForestClassifier(criterion = 'entropy', bootstrap = True) 
clf_grid_search = GridSearchCV(estimator = clf_grid,
                               param_grid = param_grid,
                               cv = 5,
                               n_jobs = 4,
                               verbose=2)
clf_grid_search.fit(X,y)

## Use this ML model below
### Parameters from GridSearchCV
- 'n_estimators': 307,
- 'min_samples_split': 15,
- 'min_samples_leaf': 3,
- 'max_depth': 20,
- 'criterion': 'entropy',
- 'bootstrap': True}

In [10]:
param_grid = {
    'n_estimators': [307],
    'max_depth': [20],  
    'min_samples_split': [15],  
    'min_samples_leaf': [3]}

In [11]:
%%time
clf_grid = RandomForestClassifier(criterion = 'entropy', bootstrap = True) 
clf_grid_search = GridSearchCV(estimator = clf_grid,
                               param_grid = param_grid,
                               cv = 5,
                               n_jobs = 4,
                               verbose=2)
clf_grid_search.fit(X,y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Wall time: 9min 51s


GridSearchCV(cv=5, estimator=RandomForestClassifier(criterion='entropy'),
             n_jobs=4,
             param_grid={'max_depth': [20], 'min_samples_leaf': [3],
                         'min_samples_split': [15], 'n_estimators': [307]},
             verbose=2)

In [12]:
%%time
resultados = cross_val_predict(clf_grid_search, X, y, cv = 5)
print(metrics.accuracy_score(y, resultados))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
0.9422848897841398
Wall time: 50min 49s


In [14]:
%%time
print(metrics.classification_report(y, resultados))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97     62867
           1       0.78      0.60      0.68      7132

    accuracy                           0.94     69999
   macro avg       0.87      0.79      0.82     69999
weighted avg       0.94      0.94      0.94     69999

Wall time: 172 ms


## Predict test dataset with the first ML model

In [15]:
test = pd.read_csv('test.csv')
test.head()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2t_mou_8,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2m_mou_8,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2f_mou_8,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_t2c_mou_8,loc_og_mou_6,loc_og_mou_7,loc_og_mou_8,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2t_mou_8,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2m_mou_8,std_og_t2f_mou_6,std_og_t2f_mou_7,std_og_t2f_mou_8,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_t2c_mou_8,std_og_mou_6,std_og_mou_7,std_og_mou_8,isd_og_mou_6,isd_og_mou_7,isd_og_mou_8,spl_og_mou_6,spl_og_mou_7,spl_og_mou_8,og_others_6,og_others_7,og_others_8,total_og_mou_6,total_og_mou_7,total_og_mou_8,loc_ic_t2t_mou_6,loc_ic_t2t_mou_7,loc_ic_t2t_mou_8,loc_ic_t2m_mou_6,loc_ic_t2m_mou_7,loc_ic_t2m_mou_8,loc_ic_t2f_mou_6,loc_ic_t2f_mou_7,loc_ic_t2f_mou_8,loc_ic_mou_6,loc_ic_mou_7,loc_ic_mou_8,std_ic_t2t_mou_6,std_ic_t2t_mou_7,std_ic_t2t_mou_8,std_ic_t2m_mou_6,std_ic_t2m_mou_7,std_ic_t2m_mou_8,std_ic_t2f_mou_6,std_ic_t2f_mou_7,std_ic_t2f_mou_8,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_t2o_mou_8,std_ic_mou_6,std_ic_mou_7,std_ic_mou_8,total_ic_mou_6,total_ic_mou_7,total_ic_mou_8,spl_ic_mou_6,spl_ic_mou_7,spl_ic_mou_8,isd_ic_mou_6,isd_ic_mou_7,isd_ic_mou_8,ic_others_6,ic_others_7,ic_others_8,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8,total_rech_data_6,total_rech_data_7,total_rech_data_8,max_rech_data_6,max_rech_data_7,max_rech_data_8,count_rech_2g_6,count_rech_2g_7,count_rech_2g_8,count_rech_3g_6,count_rech_3g_7,count_rech_3g_8,av_rech_amt_data_6,av_rech_amt_data_7,av_rech_amt_data_8,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,arpu_3g_6,arpu_3g_7,arpu_3g_8,arpu_2g_6,arpu_2g_7,arpu_2g_8,night_pck_user_6,night_pck_user_7,night_pck_user_8,monthly_2g_6,monthly_2g_7,monthly_2g_8,sachet_2g_6,sachet_2g_7,sachet_2g_8,monthly_3g_6,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g
0,69999,109,0.00000000,0.00000000,0.00000000,6/30/2014,7/31/2014,8/31/2014,91.88200000,65.33000000,64.44500000,31.78000000,20.23000000,23.11000000,60.16000000,32.16000000,34.83000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,24.88000000,20.23000000,21.06000000,18.13000000,10.89000000,8.36000000,0.00000000,13.58000000,0.00000000,0.00000000,0.00000000,0.03000000,43.01000000,44.71000000,29.43000000,6.90000000,0.00000000,2.05000000,42.03000000,7.68000000,26.43000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,48.93000000,7.68000000,28.48000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.03000000,0.00000000,0.00000000,0.00000000,91.94000000,52.39000000,57.94000000,30.33000000,37.56000000,21.98000000,10.21000000,4.59000000,9.53000000,0.26000000,0.00000000,0.00000000,40.81000000,42.16000000,31.51000000,0.00000000,0.00000000,0.00000000,0.36000000,1.04000000,4.34000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.36000000,1.04000000,4.34000000,41.73000000,43.56000000,36.26000000,0.54000000,0.34000000,0.39000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,5,5,4,103,90,60,50,30,30,6/21/2014,7/26/2014,8/24/2014,30,30,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,1692,0.00000000,0.00000000

In [16]:
test_ = test.set_index('id')[features_names].fillna(test.mean())
test_.head()

,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2t_mou_8,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2m_mou_8,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2f_mou_8,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_t2c_mou_8,loc_og_mou_6,loc_og_mou_7,loc_og_mou_8,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2t_mou_8,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2m_mou_8,std_og_t2f_mou_6,std_og_t2f_mou_7,std_og_t2f_mou_8,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_t2c_mou_8,std_og_mou_6,std_og_mou_7,std_og_mou_8,isd_og_mou_6,isd_og_mou_7,isd_og_mou_8,spl_og_mou_6,spl_og_mou_7,spl_og_mou_8,og_others_6,og_others_7,og_others_8,total_og_mou_6,total_og_mou_7,total_og_mou_8,loc_ic_t2t_mou_6,loc_ic_t2t_mou_7,loc_ic_t2t_mou_8,loc_ic_t2m_mou_6,loc_ic_t2m_mou_7,loc_ic_t2m_mou_8,loc_ic_t2f_mou_6,loc_ic_t2f_mou_7,loc_ic_t2f_mou_8,loc_ic_mou_6,loc_ic_mou_7,loc_ic_mou_8,std_ic_t2t_mou_6,std_ic_t2t_mou_7,std_ic_t2t_mou_8,std_ic_t2m_mou_6,std_ic_t2m_mou_7,std_ic_t2m_mou_8,std_ic_t2f_mou_6,std_ic_t2f_mou_7,std_ic_t2f_mou_8,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_t2o_mou_8,std_ic_mou_6,std_ic_mou_7,std_ic_mou_8,total_ic_mou_6,total_ic_mou_7,total_ic_mou_8,spl_ic_mou_6,spl_ic_mou_7,spl_ic_mou_8,isd_ic_mou_6,isd_ic_mou_7,isd_ic_mou_8,ic_others_6,ic_others_7,ic_others_8,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,total_rech_data_6,total_rech_data_7,total_rech_data_8,max_rech_data_6,max_rech_data_7,max_rech_data_8,count_rech_2g_6,count_rech_2g_7,count_rech_2g_8,count_rech_3g_6,count_rech_3g_7,count_rech_3g_8,av_rech_amt_data_6,av_rech_amt_data_7,av_rech_amt_data_8,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,arpu_3g_6,arpu_3g_7,arpu_3g_8,arpu_2g_6,arpu_2g_7,arpu_2g_8,night_pck_user_6,night_pck_user_7,night_pck_user_8,monthly_2g_6,monthly_2g_7,monthly_2g_8,sachet_2g_6,sachet_2g_7,sachet_2g_8,monthly_3g_6,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69999,0.00000000,0.00000000,0.00000000,91.88200000,65.33000000,64.44500000,31.78000000,20.23000000,23.11000000,60.16000000,32.16000000,34.83000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,24.88000000,20.23000000,21.06000000,18.13000000,10.89000000,8.36000000,0.00000000,13.58000000,0.00000000,0.00000000,0.00000000,0.03000000,43.01000000,44.71000000,29.43000000,6.90000000,0.00000000,2.05000000,42.03000000,7.68000000,26.43000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,48.93000000,7.68000000,28.48000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.03000000,0.00000000,0.00000000,0.00000000,91.94000000,52.39000000,57.94000000,30.33000000,37.56000000,21.98000000,10.21000000,4.59000000,9.53000000,0.26000000,0.00000000,0.00000000,40.81000000,42.16000000,31.51000000,0.00000000,0.00000000,0.00000000,0.36000000,1.04000000,4.34000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.36000000,1.04000000,4.34000000,41.73000000,43.56000000,36.26000000,0.54000000,0.34000000,0.39000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,5,5,4,103,90,60,50,30,30,30,30,0,2.45497693,2.63495977,2.65097198,126.14647330,127.48845056,126.51325423,1.86315096,2.01777835,2.01691492,0.59182597,0.61718142,0.63405706,192.06800396,199.88090319,199.17870992,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,88.36253131,90.00624968,92.46477152,85.31891628,86.07296652,87.18317470,0.02465392,0.02063327,0.02044938,0,

In [18]:
X_test = scaler.fit_transform(test_)

In [19]:
X_test

array([[ 0.        ,  0.        ,  0.        , ..., -0.25929959,
        -0.24734668, -0.24336869],
       [ 0.        ,  0.        ,  0.        , ..., -0.25929959,
        -0.24734668, -0.24336869],
       [ 0.        ,  0.        ,  0.        , ...,  1.73575652,
         2.47345839,  0.73934447],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., -0.25929959,
        -0.24734668, -0.24336869],
       [ 0.        ,  0.        ,  0.        , ..., -0.25929959,
        -0.24734668, -0.24336869],
       [ 0.        ,  0.        ,  0.        , ...,  2.63510887,
         2.21774053, -0.15088378]])

In [20]:
%%time
preds = clf_grid_search.predict(X_test)
preds

Wall time: 1.91 s


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [21]:
preds_ = pd.DataFrame(preds, columns=['churn_probability'])
preds_.head()

,churn_probability
0,0
1,0
2,0
3,0
4,0


In [22]:
test_ = test_.reset_index(level=0)
test_.head()

,id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2t_mou_8,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2m_mou_8,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2f_mou_8,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_t2c_mou_8,loc_og_mou_6,loc_og_mou_7,loc_og_mou_8,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2t_mou_8,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2m_mou_8,std_og_t2f_mou_6,std_og_t2f_mou_7,std_og_t2f_mou_8,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_t2c_mou_8,std_og_mou_6,std_og_mou_7,std_og_mou_8,isd_og_mou_6,isd_og_mou_7,isd_og_mou_8,spl_og_mou_6,spl_og_mou_7,spl_og_mou_8,og_others_6,og_others_7,og_others_8,total_og_mou_6,total_og_mou_7,total_og_mou_8,loc_ic_t2t_mou_6,loc_ic_t2t_mou_7,loc_ic_t2t_mou_8,loc_ic_t2m_mou_6,loc_ic_t2m_mou_7,loc_ic_t2m_mou_8,loc_ic_t2f_mou_6,loc_ic_t2f_mou_7,loc_ic_t2f_mou_8,loc_ic_mou_6,loc_ic_mou_7,loc_ic_mou_8,std_ic_t2t_mou_6,std_ic_t2t_mou_7,std_ic_t2t_mou_8,std_ic_t2m_mou_6,std_ic_t2m_mou_7,std_ic_t2m_mou_8,std_ic_t2f_mou_6,std_ic_t2f_mou_7,std_ic_t2f_mou_8,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_t2o_mou_8,std_ic_mou_6,std_ic_mou_7,std_ic_mou_8,total_ic_mou_6,total_ic_mou_7,total_ic_mou_8,spl_ic_mou_6,spl_ic_mou_7,spl_ic_mou_8,isd_ic_mou_6,isd_ic_mou_7,isd_ic_mou_8,ic_others_6,ic_others_7,ic_others_8,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,total_rech_data_6,total_rech_data_7,total_rech_data_8,max_rech_data_6,max_rech_data_7,max_rech_data_8,count_rech_2g_6,count_rech_2g_7,count_rech_2g_8,count_rech_3g_6,count_rech_3g_7,count_rech_3g_8,av_rech_amt_data_6,av_rech_amt_data_7,av_rech_amt_data_8,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,arpu_3g_6,arpu_3g_7,arpu_3g_8,arpu_2g_6,arpu_2g_7,arpu_2g_8,night_pck_user_6,night_pck_user_7,night_pck_user_8,monthly_2g_6,monthly_2g_7,monthly_2g_8,sachet_2g_6,sachet_2g_7,sachet_2g_8,monthly_3g_6,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g
0,69999,0.00000000,0.00000000,0.00000000,91.88200000,65.33000000,64.44500000,31.78000000,20.23000000,23.11000000,60.16000000,32.16000000,34.83000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,24.88000000,20.23000000,21.06000000,18.13000000,10.89000000,8.36000000,0.00000000,13.58000000,0.00000000,0.00000000,0.00000000,0.03000000,43.01000000,44.71000000,29.43000000,6.90000000,0.00000000,2.05000000,42.03000000,7.68000000,26.43000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,48.93000000,7.68000000,28.48000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.03000000,0.00000000,0.00000000,0.00000000,91.94000000,52.39000000,57.94000000,30.33000000,37.56000000,21.98000000,10.21000000,4.59000000,9.53000000,0.26000000,0.00000000,0.00000000,40.81000000,42.16000000,31.51000000,0.00000000,0.00000000,0.00000000,0.36000000,1.04000000,4.34000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.36000000,1.04000000,4.34000000,41.73000000,43.56000000,36.26000000,0.54000000,0.34000000,0.39000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,5,5,4,103,90,60,50,30,30,30,30,0,2.45497693,2.63495977,2.65097198,126.14647330,127.48845056,126.51325423,1.86315096,2.01777835,2.01691492,0.59182597,0.61718142,0.63405706,192.06800396,199.88090319,199.17870992,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,88.36253131,90.00624968,92.46477152,85.31891628,86.07296652,87.18317470,0.02465392,0.02063327,0.02044938,0,0,0,0,0,0,0,0,0,0,0,0,0.90995386,0.90695562,0.89194648,1692,0.00000000,0.00000000,0.00000000
1,70000,0.00000000,0.00000000,0.00000000,414.16800000,515.5680000

In [23]:
sub08 = pd.concat([test_['id'],preds_], axis=1)
sub08.head()

,id,churn_probability
0,69999,0
1,70000,0
2,70001,0
3,70002,0
4,70003,0


In [24]:
sub08.to_csv('sub08.csv', sep=',', index=False)